# Lesson A4: Macros, Unsafe & Performance

**Duration**: 210-240 minutes  
**Stage**: Advanced (Mastery)  
**Prerequisites**: All previous Advanced lessons (A1-A3)

---

## 📋 What You'll Learn

This comprehensive lesson covers three advanced topics: macros for metaprogramming, unsafe Rust for low-level control, and performance optimization techniques. You'll learn when and how to use these powerful tools while maintaining Rust's safety guarantees.

**Why this matters**: Macros enable code generation and DSLs used throughout the Rust ecosystem. Unsafe code is necessary for FFI, low-level optimizations, and implementing safe abstractions. Performance optimization skills are essential for systems programming, game development, and high-performance computing.

---

## 🎯 Learning Objectives

By the end of this lesson, you will be able to:
1. Write declarative macros with macro_rules!
2. Understand procedural macros
3. Use unsafe code blocks safely
4. Work with raw pointers and FFI (Foreign Function Interface)
5. Profile and benchmark Rust code effectively
6. Understand zero-cost abstractions and optimization
7. Apply performance best practices
8. Balance safety and performance


In [ ]:
// Basic declarative macros with macro_rules!

// Simple function-like macro
macro_rules! say_hello {
    () => {
        println!("Hello from macro!");
    };
}

// Macro with parameters
macro_rules! greet {
    ($name:expr) => {
        println!("Hello, {}!", $name);
    };
}

// Macro with multiple patterns
macro_rules! calculate {
    // Addition
    ($a:expr + $b:expr) => {
        $a + $b
    };
    // Multiplication
    ($a:expr * $b:expr) => {
        $a * $b
    };
    // Power (using repetition)
    ($base:expr ^ $exp:expr) => {
        (0..$exp).fold(1, |acc, _| acc * $base)
    };
}

// Macro with repetition
macro_rules! create_function {
    ($func_name:ident, $($param:ident: $param_type:ty),*) => {
        fn $func_name($($param: $param_type),*) {
            println!("Function {} called with parameters:", stringify!($func_name));
            $(
                println!("  {}: {:?}", stringify!($param), $param);
            )*
        }
    };
}

// Macro for creating structs with debug info
macro_rules! debug_struct {
    ($name:ident { $($field:ident: $field_type:ty),* $(,)? }) => {
        #[derive(Debug, Clone)]
        struct $name {
            $($field: $field_type),*
        }
        
        impl $name {
            fn new($($field: $field_type),*) -> Self {
                $name {
                    $($field),*
                }
            }
            
            fn describe(&self) {
                println!("Struct {}: {:?}", stringify!($name), self);
                $(
                    println!("  {}: {:?}", stringify!($field), self.$field);
                )*
            }
        }
    };
}

// Macro for creating enums with utility methods
macro_rules! smart_enum {
    ($name:ident { $($variant:ident),* $(,)? }) => {
        #[derive(Debug, Clone, PartialEq)]
        enum $name {
            $($variant),*
        }
        
        impl $name {
            fn all_variants() -> Vec<$name> {
                vec![$($name::$variant),*]
            }
            
            fn variant_count() -> usize {
                let mut count = 0;
                $(count += 1; let _ = $name::$variant;)*
                count
            }
            
            fn name(&self) -> &'static str {
                match self {
                    $($name::$variant => stringify!($variant)),*
                }
            }
        }
    };
}

fn basic_macros_demo() {
    println!("=== Basic Declarative Macros ===");
    
    // Simple macros
    say_hello!();
    greet!("Rust Developer");
    
    // Calculation macro
    println!("5 + 3 = {}", calculate!(5 + 3));
    println!("4 * 7 = {}", calculate!(4 * 7));
    println!("2 ^ 5 = {}", calculate!(2 ^ 5));
    
    // Generated function
    create_function!(test_func, x: i32, y: String, z: bool);
    test_func(42, "hello".to_string(), true);
    
    // Generated struct
    debug_struct!(Person {
        name: String,
        age: u32,
        active: bool,
    });
    
    let person = Person::new("Alice".to_string(), 30, true);
    person.describe();
    
    // Generated enum
    smart_enum!(Color {
        Red,
        Green,
        Blue,
        Yellow,
    });
    
    println!("\nColor enum has {} variants", Color::variant_count());
    println!("All colors: {:?}", Color::all_variants());
    
    let color = Color::Blue;
    println!("Selected color: {} ({:?})", color.name(), color);
}

basic_macros_demo();

In [ ]:
// Advanced macro patterns and techniques

// Macro for creating a hash map with syntax sugar
macro_rules! hashmap {
    // Empty map
    () => {
        std::collections::HashMap::new()
    };
    // Map with key-value pairs
    ($($key:expr => $value:expr),* $(,)?) => {
        {
            let mut map = std::collections::HashMap::new();
            $(
                map.insert($key, $value);
            )*
            map
        }
    };
}

// Macro for creating a builder pattern
macro_rules! builder {
    ($name:ident {
        $($field:ident: $field_type:ty),* $(,)?
    }) => {
        #[derive(Debug, Default)]
        struct $name {
            $($field: Option<$field_type>),*
        }
        
        paste::paste! {
            impl $name {
                fn new() -> Self {
                    Self::default()
                }
                
                $(
                    fn [<with_ $field>](mut self, $field: $field_type) -> Self {
                        self.$field = Some($field);
                        self
                    }
                    
                    fn [<set_ $field>](&mut self, $field: $field_type) {
                        self.$field = Some($field);
                    }
                    
                    fn [<get_ $field>](&self) -> Option<&$field_type> {
                        self.$field.as_ref()
                    }
                )*
                
                fn build(self) -> Result<[<$name Built>], String> {
                    $(
                        let $field = self.$field.ok_or_else(|| {
                            format!("Missing required field: {}", stringify!($field))
                        })?;
                    )*
                    
                    Ok([<$name Built>] {
                        $($field),*
                    })
                }
            }
        }
        
        paste::paste! {
            #[derive(Debug)]
            struct [<$name Built>] {
                $($field: $field_type),*
            }
        }
    };
}

// Note: The paste crate is not available in evcxr, so let's create a simpler version
macro_rules! simple_builder {
    ($name:ident {
        $($field:ident: $field_type:ty),* $(,)?
    }) => {
        #[derive(Debug, Default)]
        struct $name {
            $($field: Option<$field_type>),*
        }
        
        impl $name {
            fn new() -> Self {
                Self::default()
            }
            
            $(
                fn $field(mut self, value: $field_type) -> Self {
                    self.$field = Some(value);
                    self
                }
            )*
            
            fn build(self) -> Result<String, String> {
                let mut result = format!("{}Builder {{", stringify!($name));
                $(
                    if let Some(value) = self.$field {
                        result.push_str(&format!(" {}: {:?},", stringify!($field), value));
                    } else {
                        return Err(format!("Missing field: {}", stringify!($field)));
                    }
                )*
                result.push_str(" }");
                Ok(result)
            }
        }
    };
}

// Macro for creating test cases
macro_rules! test_cases {
    ($test_name:ident: $($case:expr => $expected:expr),* $(,)?) => {
        fn $test_name() {
            println!("Running test: {}", stringify!($test_name));
            let mut passed = 0;
            let mut total = 0;
            
            $(
                total += 1;
                let result = $case;
                if result == $expected {
                    println!("  ✅ Case {}: {} == {} (PASS)", total, result, $expected);
                    passed += 1;
                } else {
                    println!("  ❌ Case {}: {} != {} (FAIL)", total, result, $expected);
                }
            )*
            
            println!("  Result: {}/{} tests passed\n", passed, total);
        }
    };
}

// Macro for creating a simple DSL
macro_rules! query {
    (SELECT $($field:ident),* FROM $table:ident WHERE $condition:expr) => {
        {
            let fields = vec![$(stringify!($field)),*];
            let table = stringify!($table);
            let condition = stringify!($condition);
            
            format!("SQL Query: SELECT {} FROM {} WHERE {}", 
                   fields.join(", "), table, condition)
        }
    };
    (INSERT INTO $table:ident VALUES ($($value:expr),*)) => {
        {
            let table = stringify!($table);
            let values = vec![$(format!("{:?}", $value)),*];
            
            format!("SQL Query: INSERT INTO {} VALUES ({})", 
                   table, values.join(", "))
        }
    };
}

// Macro for logging with levels
macro_rules! log {
    (DEBUG: $($arg:tt)*) => {
        println!("[DEBUG] {}", format!($($arg)*));
    };
    (INFO: $($arg:tt)*) => {
        println!("[INFO]  {}", format!($($arg)*));
    };
    (WARN: $($arg:tt)*) => {
        println!("[WARN]  {}", format!($($arg)*));
    };
    (ERROR: $($arg:tt)*) => {
        println!("[ERROR] {}", format!($($arg)*));
    };
}

fn advanced_macros_demo() {
    println!("\n=== Advanced Macro Patterns ===");
    
    // HashMap creation
    let map = hashmap!{
        "name" => "Alice",
        "age" => "30",
        "city" => "New York",
    };
    println!("Created map: {:?}", map);
    
    // Builder pattern
    simple_builder!(Config {
        host: String,
        port: u16,
        ssl: bool,
    });
    
    let config = Config::new()
        .host("localhost".to_string())
        .port(8080)
        .ssl(true)
        .build();
    
    match config {
        Ok(config_str) => println!("Built config: {}", config_str),
        Err(e) => println!("Config error: {}", e),
    }
    
    // Test cases
    test_cases!(math_tests:
        2 + 2 => 4,
        3 * 4 => 12,
        10 / 2 => 5,
        5 - 1 => 4,
    );
    
    math_tests();
    
    // DSL for SQL-like queries
    let query1 = query!(SELECT name, age FROM users WHERE age > 18);
    let query2 = query!(INSERT INTO users VALUES ("Bob", 25, "Engineer"));
    
    println!("Generated queries:");
    println!("  {}", query1);
    println!("  {}", query2);
    
    // Logging macro
    log!(DEBUG: "Debug message with value: {}", 42);
    log!(INFO: "Application started successfully");
    log!(WARN: "Low memory warning: {}MB remaining", 128);
    log!(ERROR: "Failed to connect to database: {}", "timeout");
}

advanced_macros_demo();

In [ ]:
// Macro debugging and best practices

// Macro with debugging capabilities
macro_rules! debug_macro {
    ($($input:tt)*) => {
        {
            println!("Macro input: {}", stringify!($($input)*));
            let result = { $($input)* };
            println!("Macro result: {:?}", result);
            result
        }
    };
}

// Macro with error handling
macro_rules! safe_divide {
    ($a:expr, $b:expr) => {
        {
            let dividend = $a;
            let divisor = $b;
            
            if divisor == 0 {
                Err(format!("Division by zero: {} / {}", dividend, divisor))
            } else {
                Ok(dividend / divisor)
            }
        }
    };
}

// Macro with hygiene demonstration
macro_rules! hygiene_test {
    ($x:expr) => {
        {
            let temp = $x * 2; // This 'temp' is hygienic
            temp + 1
        }
    };
}

// Macro for creating documentation
macro_rules! documented_function {
    (
        $(#[doc = $doc:expr])*
        fn $name:ident($($param:ident: $param_type:ty),*) -> $return_type:ty {
            $($body:tt)*
        }
    ) => {
        $(
            #[doc = $doc]
        )*
        fn $name($($param: $param_type),*) -> $return_type {
            println!("Calling function: {}", stringify!($name));
            $($body)*
        }
    };
}

// Macro for performance measurement
macro_rules! time_it {
    ($name:expr, $code:block) => {
        {
            let start = std::time::Instant::now();
            let result = $code;
            let duration = start.elapsed();
            println!("⏱️  {} took: {:?}", $name, duration);
            result
        }
    };
}

// Macro for creating configuration structs
macro_rules! config_struct {
    (
        $name:ident {
            $(
                $field:ident: $field_type:ty = $default:expr,
            )*
        }
    ) => {
        #[derive(Debug, Clone)]
        struct $name {
            $($field: $field_type),*
        }
        
        impl Default for $name {
            fn default() -> Self {
                $name {
                    $($field: $default),*
                }
            }
        }
        
        impl $name {
            fn new() -> Self {
                Self::default()
            }
            
            $(
                fn $field(mut self, value: $field_type) -> Self {
                    self.$field = value;
                    self
                }
            )*
            
            fn validate(&self) -> Result<(), String> {
                // Add validation logic here if needed
                Ok(())
            }
            
            fn summary(&self) {
                println!("Configuration {}:", stringify!($name));
                $(
                    println!("  {}: {:?}", stringify!($field), self.$field);
                )*
            }
        }
    };
}

// Macro for creating state machines
macro_rules! state_machine {
    (
        $name:ident {
            states: [$($state:ident),*],
            initial: $initial:ident,
            transitions: {
                $(
                    $from:ident -> $to:ident on $event:ident,
                )*
            }
        }
    ) => {
        #[derive(Debug, Clone, PartialEq)]
        enum State {
            $($state),*
        }
        
        #[derive(Debug, Clone)]
        enum Event {
            $($event),*
        }
        
        #[derive(Debug)]
        struct $name {
            current_state: State,
        }
        
        impl $name {
            fn new() -> Self {
                $name {
                    current_state: State::$initial,
                }
            }
            
            fn current_state(&self) -> &State {
                &self.current_state
            }
            
            fn handle_event(&mut self, event: Event) -> Result<(), String> {
                let old_state = self.current_state.clone();
                
                match (&self.current_state, &event) {
                    $(
                        (State::$from, Event::$event) => {
                            self.current_state = State::$to;
                            println!("Transition: {:?} -> {:?} on {:?}", 
                                   old_state, self.current_state, event);
                            Ok(())
                        }
                    )*
                    _ => {
                        Err(format!("Invalid transition: {:?} on {:?}", 
                               self.current_state, event))
                    }
                }
            }
        }
    };
}

fn macro_debugging_demo() {
    println!("\n=== Macro Debugging and Best Practices ===");
    
    // Debug macro usage
    let result = debug_macro!(5 + 3 * 2);
    println!("Final result: {}\n", result);
    
    // Safe division
    match safe_divide!(10, 2) {
        Ok(result) => println!("Division result: {}", result),
        Err(e) => println!("Division error: {}", e),
    }
    
    match safe_divide!(10, 0) {
        Ok(result) => println!("Division result: {}", result),
        Err(e) => println!("Division error: {}", e),
    }
    
    // Hygiene test
    let temp = 5; // This won't conflict with macro's temp
    let result = hygiene_test!(temp);
    println!("\nHygiene test: temp = {}, result = {}", temp, result);
    
    // Documented function
    documented_function!(
        #[doc = "Calculates the square of a number"]
        #[doc = "Returns the input multiplied by itself"]
        fn square(x: i32) -> i32 {
            x * x
        }
    );
    
    let squared = square(7);
    println!("Square of 7: {}", squared);
    
    // Performance measurement
    let result = time_it!("Fibonacci calculation", {
        fn fib(n: u32) -> u32 {
            match n {
                0 | 1 => n,
                _ => fib(n - 1) + fib(n - 2),
            }
        }
        fib(20)
    });
    println!("Fibonacci result: {}", result);
    
    // Configuration struct
    config_struct!(ServerConfig {
        host: String = "localhost".to_string(),
        port: u16 = 8080,
        workers: usize = 4,
        debug: bool = false,
    });
    
    let config = ServerConfig::new()
        .host("0.0.0.0".to_string())
        .port(3000)
        .workers(8)
        .debug(true);
    
    config.summary();
    
    // State machine
    state_machine!(TrafficLight {
        states: [Red, Yellow, Green],
        initial: Red,
        transitions: {
            Red -> Green on Go,
            Green -> Yellow on Caution,
            Yellow -> Red on Stop,
        }
    });
    
    let mut light = TrafficLight::new();
    println!("\nTraffic light initial state: {:?}", light.current_state());
    
    let _ = light.handle_event(Event::Go);
    let _ = light.handle_event(Event::Caution);
    let _ = light.handle_event(Event::Stop);
    
    // Try invalid transition
    if let Err(e) = light.handle_event(Event::Caution) {
        println!("Error: {}", e);
    }
}

macro_debugging_demo();

In [ ]:
// TODO: Complete the testing framework macro

// Testing framework with multiple assertion types
macro_rules! test_framework {
    (
        suite $suite_name:ident {
            $(
                test $test_name:ident {
                    $($test_body:tt)*
                }
            )*
        }
    ) => {
        mod $suite_name {
            use super::*;
            
            #[derive(Debug)]
            struct TestResult {
                name: &'static str,
                passed: bool,
                message: String,
            }
            
            impl TestResult {
                fn new(name: &'static str, passed: bool, message: String) -> Self {
                    TestResult { name, passed, message }
                }
            }
            
            $(
                fn $test_name() -> TestResult {
                    println!("  Running test: {}", stringify!($test_name));
                    
                    // Capture test execution
                    let result = std::panic::catch_unwind(|| {
                        $($test_body)*
                    });
                    
                    match result {
                        Ok(_) => TestResult::new(
                            stringify!($test_name),
                            true,
                            "Test passed".to_string()
                        ),
                        Err(e) => {
                            let message = if let Some(s) = e.downcast_ref::<&str>() {
                                s.to_string()
                            } else if let Some(s) = e.downcast_ref::<String>() {
                                s.clone()
                            } else {
                                "Test panicked".to_string()
                            };
                            TestResult::new(
                                stringify!($test_name),
                                false,
                                message
                            )
                        }
                    }
                }
            )*
            
            pub fn run_all_tests() {
                println!("\n🧪 Running test suite: {}", stringify!($suite_name));
                
                let mut results = Vec::new();
                
                $(
                    results.push($test_name());
                )*
                
                // Print results
                let total = results.len();
                let passed = results.iter().filter(|r| r.passed).count();
                let failed = total - passed;
                
                println!("\n📊 Test Results:");
                for result in &results {
                    let status = if result.passed { "✅ PASS" } else { "❌ FAIL" };
                    println!("  {} {}: {}", status, result.name, result.message);
                }
                
                println!("\n📈 Summary: {}/{} tests passed ({:.1}%)", 
                        passed, total, (passed as f64 / total as f64) * 100.0);
                
                if failed > 0 {
                    println!("❌ {} test(s) failed", failed);
                } else {
                    println!("🎉 All tests passed!");
                }
            }
        }
    };
}

// Assertion macros for the testing framework
macro_rules! assert_eq_msg {
    ($left:expr, $right:expr) => {
        {
            let left_val = $left;
            let right_val = $right;
            if left_val != right_val {
                panic!("Assertion failed: {} != {} (left: {:?}, right: {:?})", 
                      stringify!($left), stringify!($right), left_val, right_val);
            }
        }
    };
}

macro_rules! assert_true {
    ($expr:expr) => {
        {
            let val = $expr;
            if !val {
                panic!("Assertion failed: {} is not true (value: {:?})", 
                      stringify!($expr), val);
            }
        }
    };
}

macro_rules! assert_false {
    ($expr:expr) => {
        {
            let val = $expr;
            if val {
                panic!("Assertion failed: {} is not false (value: {:?})", 
                      stringify!($expr), val);
            }
        }
    };
}

macro_rules! assert_contains {
    ($container:expr, $item:expr) => {
        {
            let container = &$container;
            let item = &$item;
            if !container.contains(item) {
                panic!("Assertion failed: {:?} does not contain {:?}", 
                      container, item);
            }
        }
    };
}

// Benchmark macro
macro_rules! benchmark {
    ($name:expr, $iterations:expr, $code:block) => {
        {
            println!("\n⏱️  Benchmarking: {} ({} iterations)", $name, $iterations);
            
            let start = std::time::Instant::now();
            
            for _ in 0..$iterations {
                let _ = $code;
            }
            
            let total_duration = start.elapsed();
            let avg_duration = total_duration / $iterations;
            
            println!("  Total time: {:?}", total_duration);
            println!("  Average time per iteration: {:?}", avg_duration);
            println!("  Iterations per second: {:.0}", 
                    1_000_000_000.0 / avg_duration.as_nanos() as f64);
            
            (total_duration, avg_duration)
        }
    };
}

// Property-based testing macro
macro_rules! property_test {
    ($name:ident: $property:expr, $generator:expr, $iterations:expr) => {
        fn $name() {
            println!("\n🔍 Property test: {}", stringify!($name));
            
            let mut passed = 0;
            let mut failed = 0;
            
            for i in 0..$iterations {
                let test_value = $generator(i);
                
                if $property(test_value) {
                    passed += 1;
                } else {
                    failed += 1;
                    println!("  ❌ Property failed for input: {:?}", test_value);
                }
            }
            
            println!("  Results: {}/{} passed ({:.1}%)", 
                    passed, $iterations, (passed as f64 / $iterations as f64) * 100.0);
            
            if failed == 0 {
                println!("  ✅ Property holds for all test cases");
            } else {
                println!("  ❌ Property failed for {} test cases", failed);
            }
        }
    };
}

// Example functions to test
fn add(a: i32, b: i32) -> i32 {
    a + b
}

fn multiply(a: i32, b: i32) -> i32 {
    a * b
}

fn is_even(n: i32) -> bool {
    n % 2 == 0
}

fn factorial(n: u32) -> u32 {
    match n {
        0 | 1 => 1,
        _ => n * factorial(n - 1),
    }
}

// Create test suites using the framework
test_framework!(
    suite math_tests {
        test addition_works {
            assert_eq_msg!(add(2, 3), 5);
            assert_eq_msg!(add(-1, 1), 0);
            assert_eq_msg!(add(0, 0), 0);
        }
        
        test multiplication_works {
            assert_eq_msg!(multiply(3, 4), 12);
            assert_eq_msg!(multiply(-2, 5), -10);
            assert_eq_msg!(multiply(0, 100), 0);
        }
        
        test even_number_detection {
            assert_true!(is_even(2));
            assert_true!(is_even(0));
            assert_false!(is_even(1));
            assert_false!(is_even(3));
        }
        
        test factorial_calculation {
            assert_eq_msg!(factorial(0), 1);
            assert_eq_msg!(factorial(1), 1);
            assert_eq_msg!(factorial(5), 120);
        }
        
        test string_operations {
            let text = "Hello, World!";
            assert_contains!(text, "Hello");
            assert_contains!(text, "World");
            assert_true!(text.len() > 10);
        }
        
        test intentional_failure {
            // This test will fail to demonstrate error handling
            assert_eq_msg!(add(2, 2), 5); // This should fail
        }
    }
);

// Property-based tests
property_test!(addition_commutative: 
    |x: i32| add(x, 5) == add(5, x),
    |i| (i % 100) as i32,
    50
);

property_test!(multiplication_by_zero:
    |x: i32| multiply(x, 0) == 0,
    |i| (i % 1000) as i32 - 500,
    100
);

fn testing_framework_demo() {
    println!("\n=== Testing Framework Demo ===");
    
    // Run the test suite
    math_tests::run_all_tests();
    
    // Run property-based tests
    addition_commutative();
    multiplication_by_zero();
    
    // Benchmark examples
    benchmark!("Simple addition", 1000, {
        add(42, 58)
    });
    
    benchmark!("Factorial calculation", 100, {
        factorial(10)
    });
    
    benchmark!("String operations", 1000, {
        let s = "Hello, World!".to_string();
        s.to_uppercase()
    });
    
    println!("\n🎯 Testing framework features demonstrated:");
    println!("  ✅ Custom assertion macros");
    println!("  ✅ Test suite organization");
    println!("  ✅ Error handling and reporting");
    println!("  ✅ Property-based testing");
    println!("  ✅ Benchmarking capabilities");
    println!("  ✅ Comprehensive test statistics");
}

testing_framework_demo();

In [ ]:
// Raw pointers and basic unsafe operations

use std::ptr;

fn raw_pointers_demo() {
    println!("=== Raw Pointers and Unsafe Operations ===");
    
    // Creating raw pointers from references
    let mut x = 42;
    let r1 = &x as *const i32;  // Immutable raw pointer
    let r2 = &mut x as *mut i32; // Mutable raw pointer
    
    println!("Raw pointers created:");
    println!("  r1 (const): {:p}", r1);
    println!("  r2 (mut):   {:p}", r2);
    
    // Dereferencing raw pointers (requires unsafe)
    unsafe {
        println!("\nDereferencing raw pointers:");
        println!("  *r1 = {}", *r1);
        println!("  *r2 = {}", *r2);
        
        // Modifying through mutable raw pointer
        *r2 = 100;
        println!("  After *r2 = 100: x = {}", x);
    }
    
    // Creating raw pointers from arbitrary addresses (dangerous!)
    let address = 0x12345usize;
    let r3 = address as *const i32;
    println!("\nRaw pointer from address: {:p}", r3);
    // Note: Dereferencing this would likely crash!
    
    // Null pointers
    let null_ptr: *const i32 = ptr::null();
    let null_mut_ptr: *mut i32 = ptr::null_mut();
    
    println!("\nNull pointers:");
    println!("  null_ptr.is_null(): {}", null_ptr.is_null());
    println!("  null_mut_ptr.is_null(): {}", null_mut_ptr.is_null());
}

// Unsafe functions
unsafe fn dangerous_function() {
    println!("This is an unsafe function!");
}

// Safe wrapper around unsafe code
fn safe_wrapper(slice: &[i32], index: usize) -> Option<i32> {
    if index < slice.len() {
        unsafe {
            // We've checked bounds, so this is safe
            Some(*slice.get_unchecked(index))
        }
    } else {
        None
    }
}

// Custom unsafe trait
unsafe trait UnsafeTrait {
    fn unsafe_method(&self);
}

// Implementing unsafe trait
struct SafeStruct {
    value: i32,
}

unsafe impl UnsafeTrait for SafeStruct {
    fn unsafe_method(&self) {
        println!("Unsafe method called with value: {}", self.value);
    }
}

fn unsafe_operations_demo() {
    println!("\n=== Unsafe Functions and Traits ===");
    
    // Calling unsafe function
    unsafe {
        dangerous_function();
    }
    
    // Using safe wrapper
    let numbers = vec![1, 2, 3, 4, 5];
    println!("\nSafe wrapper results:");
    println!("  Index 2: {:?}", safe_wrapper(&numbers, 2));
    println!("  Index 10: {:?}", safe_wrapper(&numbers, 10));
    
    // Using unsafe trait
    let safe_struct = SafeStruct { value: 42 };
    safe_struct.unsafe_method(); // This is safe because we implemented it correctly
}

raw_pointers_demo();
unsafe_operations_demo();

### Memory Management and Custom Allocators

In [ ]:
// Memory management and custom data structures

use std::alloc::{alloc, dealloc, Layout};
use std::ptr;
use std::mem;

// Custom vector implementation using unsafe code
struct UnsafeVec<T> {
    ptr: *mut T,
    len: usize,
    capacity: usize,
}

impl<T> UnsafeVec<T> {
    fn new() -> Self {
        UnsafeVec {
            ptr: ptr::null_mut(),
            len: 0,
            capacity: 0,
        }
    }
    
    fn with_capacity(capacity: usize) -> Self {
        let mut vec = UnsafeVec::new();
        if capacity > 0 {
            vec.grow(capacity);
        }
        vec
    }
    
    fn grow(&mut self, new_capacity: usize) {
        if new_capacity <= self.capacity {
            return;
        }
        
        let layout = Layout::array::<T>(new_capacity).unwrap();
        
        unsafe {
            let new_ptr = if self.capacity == 0 {
                alloc(layout) as *mut T
            } else {
                // In a real implementation, we'd use realloc
                let new_ptr = alloc(layout) as *mut T;
                
                // Copy existing elements
                ptr::copy_nonoverlapping(self.ptr, new_ptr, self.len);
                
                // Deallocate old memory
                let old_layout = Layout::array::<T>(self.capacity).unwrap();
                dealloc(self.ptr as *mut u8, old_layout);
                
                new_ptr
            };
            
            self.ptr = new_ptr;
            self.capacity = new_capacity;
        }
    }
    
    fn push(&mut self, value: T) {
        if self.len == self.capacity {
            let new_capacity = if self.capacity == 0 { 1 } else { self.capacity * 2 };
            self.grow(new_capacity);
        }
        
        unsafe {
            ptr::write(self.ptr.add(self.len), value);
        }
        
        self.len += 1;
    }
    
    fn pop(&mut self) -> Option<T> {
        if self.len == 0 {
            None
        } else {
            self.len -= 1;
            unsafe {
                Some(ptr::read(self.ptr.add(self.len)))
            }
        }
    }
    
    fn get(&self, index: usize) -> Option<&T> {
        if index < self.len {
            unsafe {
                Some(&*self.ptr.add(index))
            }
        } else {
            None
        }
    }
    
    fn get_mut(&mut self, index: usize) -> Option<&mut T> {
        if index < self.len {
            unsafe {
                Some(&mut *self.ptr.add(index))
            }
        } else {
            None
        }
    }
    
    fn len(&self) -> usize {
        self.len
    }
    
    fn capacity(&self) -> usize {
        self.capacity
    }
    
    fn is_empty(&self) -> bool {
        self.len == 0
    }
}

impl<T> Drop for UnsafeVec<T> {
    fn drop(&mut self) {
        // Drop all elements
        while let Some(_) = self.pop() {}
        
        // Deallocate memory
        if self.capacity > 0 {
            unsafe {
                let layout = Layout::array::<T>(self.capacity).unwrap();
                dealloc(self.ptr as *mut u8, layout);
            }
        }
    }
}

// Union example (C-style union)
#[repr(C)]
union IntOrFloat {
    i: i32,
    f: f32,
}

impl IntOrFloat {
    fn new_int(value: i32) -> Self {
        IntOrFloat { i: value }
    }
    
    fn new_float(value: f32) -> Self {
        IntOrFloat { f: value }
    }
    
    unsafe fn as_int(&self) -> i32 {
        self.i
    }
    
    unsafe fn as_float(&self) -> f32 {
        self.f
    }
}

// Static mutable variable (requires unsafe to access)
static mut GLOBAL_COUNTER: i32 = 0;

fn increment_global_counter() {
    unsafe {
        GLOBAL_COUNTER += 1;
    }
}

fn get_global_counter() -> i32 {
    unsafe {
        GLOBAL_COUNTER
    }
}

fn memory_management_demo() {
    println!("\n=== Memory Management and Custom Data Structures ===");
    
    // Custom vector demonstration
    let mut vec = UnsafeVec::with_capacity(2);
    println!("Created UnsafeVec with capacity: {}", vec.capacity());
    
    // Push elements
    vec.push("Hello");
    vec.push("World");
    vec.push("!"); // This will trigger growth
    
    println!("\nAfter pushing 3 elements:");
    println!("  Length: {}", vec.len());
    println!("  Capacity: {}", vec.capacity());
    
    // Access elements
    for i in 0..vec.len() {
        if let Some(value) = vec.get(i) {
            println!("  vec[{}] = {}", i, value);
        }
    }
    
    // Pop elements
    while let Some(value) = vec.pop() {
        println!("Popped: {}", value);
    }
    
    println!("Vector is now empty: {}", vec.is_empty());
    
    // Union demonstration
    let int_val = IntOrFloat::new_int(42);
    let float_val = IntOrFloat::new_float(3.14);
    
    unsafe {
        println!("\nUnion demonstration:");
        println!("  int_val as int: {}", int_val.as_int());
        println!("  int_val as float: {}", int_val.as_float()); // Undefined behavior!
        println!("  float_val as float: {}", float_val.as_float());
        println!("  float_val as int: {}", float_val.as_int()); // Undefined behavior!
    }
    
    // Static mutable variable
    println!("\nGlobal counter demonstration:");
    println!("  Initial value: {}", get_global_counter());
    
    increment_global_counter();
    increment_global_counter();
    increment_global_counter();
    
    println!("  After 3 increments: {}", get_global_counter());
}

memory_management_demo();

### Foreign Function Interface (FFI)

In [ ]:
// Foreign Function Interface (FFI) examples

use std::ffi::{CStr, CString};
use std::os::raw::{c_char, c_int, c_void};

// External C functions (these would normally be in a C library)
extern "C" {
    // Standard C library functions
    fn strlen(s: *const c_char) -> usize;
    fn strcmp(s1: *const c_char, s2: *const c_char) -> c_int;
    fn malloc(size: usize) -> *mut c_void;
    fn free(ptr: *mut c_void);
}

// Rust functions that can be called from C
#[no_mangle]
pub extern "C" fn rust_add(a: c_int, b: c_int) -> c_int {
    a + b
}

#[no_mangle]
pub extern "C" fn rust_string_length(s: *const c_char) -> usize {
    if s.is_null() {
        return 0;
    }
    
    unsafe {
        CStr::from_ptr(s).to_bytes().len()
    }
}

#[no_mangle]
pub extern "C" fn rust_process_array(arr: *mut c_int, len: usize) {
    if arr.is_null() {
        return;
    }
    
    unsafe {
        let slice = std::slice::from_raw_parts_mut(arr, len);
        for item in slice {
            *item *= 2; // Double each element
        }
    }
}

// Safe wrapper for C string operations
struct CStringWrapper {
    ptr: *mut c_char,
}

impl CStringWrapper {
    fn new(s: &str) -> Result<Self, std::ffi::NulError> {
        let c_string = CString::new(s)?;
        let len = c_string.as_bytes_with_nul().len();
        
        unsafe {
            let ptr = malloc(len) as *mut c_char;
            if ptr.is_null() {
                panic!("Failed to allocate memory");
            }
            
            std::ptr::copy_nonoverlapping(
                c_string.as_ptr(),
                ptr,
                len,
            );
            
            Ok(CStringWrapper { ptr })
        }
    }
    
    fn as_ptr(&self) -> *const c_char {
        self.ptr
    }
    
    fn to_string(&self) -> Result<String, std::str::Utf8Error> {
        unsafe {
            CStr::from_ptr(self.ptr).to_str().map(|s| s.to_owned())
        }
    }
    
    fn len(&self) -> usize {
        unsafe {
            strlen(self.ptr)
        }
    }
}

impl Drop for CStringWrapper {
    fn drop(&mut self) {
        unsafe {
            free(self.ptr as *mut c_void);
        }
    }
}

// Callback function type for C
type Callback = extern "C" fn(c_int) -> c_int;

// Function that takes a callback
#[no_mangle]
pub extern "C" fn rust_apply_callback(arr: *mut c_int, len: usize, callback: Callback) {
    if arr.is_null() {
        return;
    }
    
    unsafe {
        let slice = std::slice::from_raw_parts_mut(arr, len);
        for item in slice {
            *item = callback(*item);
        }
    }
}

// Example callback functions
extern "C" fn square(x: c_int) -> c_int {
    x * x
}

extern "C" fn increment(x: c_int) -> c_int {
    x + 1
}

// Opaque type for C API
#[repr(C)]
pub struct OpaqueHandle {
    _private: [u8; 0],
}

// Internal implementation
struct InternalData {
    value: i32,
    name: String,
}

#[no_mangle]
pub extern "C" fn create_handle(value: c_int, name: *const c_char) -> *mut OpaqueHandle {
    if name.is_null() {
        return std::ptr::null_mut();
    }
    
    unsafe {
        let name_str = match CStr::from_ptr(name).to_str() {
            Ok(s) => s.to_owned(),
            Err(_) => return std::ptr::null_mut(),
        };
        
        let data = Box::new(InternalData {
            value,
            name: name_str,
        });
        
        Box::into_raw(data) as *mut OpaqueHandle
    }
}

#[no_mangle]
pub extern "C" fn get_handle_value(handle: *const OpaqueHandle) -> c_int {
    if handle.is_null() {
        return -1;
    }
    
    unsafe {
        let data = &*(handle as *const InternalData);
        data.value
    }
}

#[no_mangle]
pub extern "C" fn destroy_handle(handle: *mut OpaqueHandle) {
    if !handle.is_null() {
        unsafe {
            let _ = Box::from_raw(handle as *mut InternalData);
        }
    }
}

fn ffi_demo() {
    println!("\n=== Foreign Function Interface (FFI) ===");
    
    // Using C string functions
    let rust_string = "Hello, FFI!";
    let c_string = CString::new(rust_string).unwrap();
    
    unsafe {
        let len = strlen(c_string.as_ptr());
        println!("C strlen result: {}", len);
        println!("Rust len: {}", rust_string.len());
    }
    
    // String comparison
    let str1 = CString::new("hello").unwrap();
    let str2 = CString::new("hello").unwrap();
    let str3 = CString::new("world").unwrap();
    
    unsafe {
        let cmp1 = strcmp(str1.as_ptr(), str2.as_ptr());
        let cmp2 = strcmp(str1.as_ptr(), str3.as_ptr());
        
        println!("\nString comparison:");
        println!("  'hello' vs 'hello': {}", cmp1);
        println!("  'hello' vs 'world': {}", cmp2);
    }
    
    // C string wrapper
    let wrapper = CStringWrapper::new("Wrapped string").unwrap();
    println!("\nC string wrapper:");
    println!("  Length: {}", wrapper.len());
    println!("  Content: {}", wrapper.to_string().unwrap());
    
    // Array processing
    let mut numbers = [1, 2, 3, 4, 5];
    println!("\nArray before processing: {:?}", numbers);
    
    rust_process_array(numbers.as_mut_ptr(), numbers.len());
    println!("Array after doubling: {:?}", numbers);
    
    // Callback functions
    let mut values = [1, 2, 3, 4, 5];
    println!("\nCallback demonstration:");
    println!("  Original: {:?}", values);
    
    rust_apply_callback(values.as_mut_ptr(), values.len(), square);
    println!("  After square: {:?}", values);
    
    rust_apply_callback(values.as_mut_ptr(), values.len(), increment);
    println!("  After increment: {:?}", values);
    
    // Opaque handle
    let name = CString::new("Test Handle").unwrap();
    let handle = create_handle(42, name.as_ptr());
    
    if !handle.is_null() {
        let value = get_handle_value(handle);
        println!("\nOpaque handle value: {}", value);
        destroy_handle(handle);
        println!("Handle destroyed");
    }
    
    println!("\n💡 FFI Safety Tips:");
    println!("  - Always check for null pointers");
    println!("  - Validate string encoding (UTF-8 vs C strings)");
    println!("  - Match memory allocation/deallocation");
    println!("  - Use repr(C) for struct layout compatibility");
    println!("  - Handle errors gracefully across language boundaries");
}

ffi_demo();

In [ ]:
// TODO: Complete the safe unsafe code patterns

use std::ptr;
use std::mem;
use std::alloc::{alloc, dealloc, Layout};

// Safe ring buffer implementation using unsafe code
struct RingBuffer<T> {
    data: *mut T,
    capacity: usize,
    head: usize,
    tail: usize,
    len: usize,
}

impl<T> RingBuffer<T> {
    fn new(capacity: usize) -> Self {
        assert!(capacity > 0, "Capacity must be greater than 0");
        
        let layout = Layout::array::<T>(capacity).unwrap();
        let data = unsafe { alloc(layout) as *mut T };
        
        if data.is_null() {
            panic!("Failed to allocate memory for ring buffer");
        }
        
        RingBuffer {
            data,
            capacity,
            head: 0,
            tail: 0,
            len: 0,
        }
    }
    
    fn push(&mut self, value: T) -> Result<(), T> {
        if self.len == self.capacity {
            return Err(value); // Buffer is full
        }
        
        unsafe {
            ptr::write(self.data.add(self.tail), value);
        }
        
        self.tail = (self.tail + 1) % self.capacity;
        self.len += 1;
        
        Ok(())
    }
    
    fn pop(&mut self) -> Option<T> {
        if self.len == 0 {
            return None;
        }
        
        let value = unsafe {
            ptr::read(self.data.add(self.head))
        };
        
        self.head = (self.head + 1) % self.capacity;
        self.len -= 1;
        
        Some(value)
    }
    
    fn peek(&self) -> Option<&T> {
        if self.len == 0 {
            None
        } else {
            unsafe {
                Some(&*self.data.add(self.head))
            }
        }
    }
    
    fn len(&self) -> usize {
        self.len
    }
    
    fn capacity(&self) -> usize {
        self.capacity
    }
    
    fn is_empty(&self) -> bool {
        self.len == 0
    }
    
    fn is_full(&self) -> bool {
        self.len == self.capacity
    }
    
    // Safe iterator implementation
    fn iter(&self) -> RingBufferIter<T> {
        RingBufferIter {
            buffer: self,
            index: 0,
        }
    }
}

impl<T> Drop for RingBuffer<T> {
    fn drop(&mut self) {
        // Drop all remaining elements
        while let Some(_) = self.pop() {}
        
        // Deallocate memory
        unsafe {
            let layout = Layout::array::<T>(self.capacity).unwrap();
            dealloc(self.data as *mut u8, layout);
        }
    }
}

// Safe iterator for ring buffer
struct RingBufferIter<'a, T> {
    buffer: &'a RingBuffer<T>,
    index: usize,
}

impl<'a, T> Iterator for RingBufferIter<'a, T> {
    type Item = &'a T;
    
    fn next(&mut self) -> Option<Self::Item> {
        if self.index >= self.buffer.len {
            None
        } else {
            let actual_index = (self.buffer.head + self.index) % self.buffer.capacity;
            let item = unsafe {
                &*self.buffer.data.add(actual_index)
            };
            self.index += 1;
            Some(item)
        }
    }
}

// Memory pool for efficient allocation
struct MemoryPool<T> {
    blocks: Vec<*mut T>,
    block_size: usize,
    free_list: Vec<*mut T>,
}

impl<T> MemoryPool<T> {
    fn new(block_size: usize, initial_blocks: usize) -> Self {
        let mut pool = MemoryPool {
            blocks: Vec::new(),
            block_size,
            free_list: Vec::new(),
        };
        
        pool.allocate_block();
        
        for _ in 0..initial_blocks {
            pool.allocate_block();
        }
        
        pool
    }
    
    fn allocate_block(&mut self) {
        let layout = Layout::array::<T>(self.block_size).unwrap();
        
        unsafe {
            let block = alloc(layout) as *mut T;
            if block.is_null() {
                panic!("Failed to allocate memory block");
            }
            
            self.blocks.push(block);
            
            // Add all slots in this block to free list
            for i in 0..self.block_size {
                self.free_list.push(block.add(i));
            }
        }
    }
    
    fn allocate(&mut self) -> Option<*mut T> {
        if self.free_list.is_empty() {
            self.allocate_block();
        }
        
        self.free_list.pop()
    }
    
    fn deallocate(&mut self, ptr: *mut T) {
        // In a real implementation, we'd verify the pointer is valid
        self.free_list.push(ptr);
    }
    
    fn available_slots(&self) -> usize {
        self.free_list.len()
    }
    
    fn total_capacity(&self) -> usize {
        self.blocks.len() * self.block_size
    }
}

impl<T> Drop for MemoryPool<T> {
    fn drop(&mut self) {
        unsafe {
            let layout = Layout::array::<T>(self.block_size).unwrap();
            for block in &self.blocks {
                dealloc(*block as *mut u8, layout);
            }
        }
    }
}

// Safe wrapper for raw pointer operations
struct SafePtr<T> {
    ptr: *mut T,
    len: usize,
}

impl<T> SafePtr<T> {
    fn new(slice: &mut [T]) -> Self {
        SafePtr {
            ptr: slice.as_mut_ptr(),
            len: slice.len(),
        }
    }
    
    fn get(&self, index: usize) -> Option<&T> {
        if index < self.len {
            unsafe {
                Some(&*self.ptr.add(index))
            }
        } else {
            None
        }
    }
    
    fn get_mut(&mut self, index: usize) -> Option<&mut T> {
        if index < self.len {
            unsafe {
                Some(&mut *self.ptr.add(index))
            }
        } else {
            None
        }
    }
    
    fn len(&self) -> usize {
        self.len
    }
    
    // Unsafe but documented operation
    /// # Safety
    /// 
    /// The caller must ensure that:
    /// - `index` is less than `self.len()`
    /// - The pointer is still valid
    /// - No other mutable references exist to this element
    unsafe fn get_unchecked(&self, index: usize) -> &T {
        &*self.ptr.add(index)
    }
    
    /// # Safety
    /// 
    /// The caller must ensure that:
    /// - `index` is less than `self.len()`
    /// - The pointer is still valid
    /// - No other references exist to this element
    unsafe fn get_unchecked_mut(&mut self, index: usize) -> &mut T {
        &mut *self.ptr.add(index)
    }
}

fn safe_unsafe_patterns_demo() {
    println!("\n=== Safe Unsafe Code Patterns ===");
    
    // Ring buffer demonstration
    let mut ring = RingBuffer::new(3);
    
    println!("Ring buffer operations:");
    println!("  Initial: len={}, capacity={}", ring.len(), ring.capacity());
    
    // Fill the buffer
    ring.push("First").unwrap();
    ring.push("Second").unwrap();
    ring.push("Third").unwrap();
    
    println!("  After filling: len={}, is_full={}", ring.len(), ring.is_full());
    
    // Try to overfill
    if let Err(rejected) = ring.push("Fourth") {
        println!("  Rejected '{}' - buffer full", rejected);
    }
    
    // Iterate over contents
    print!("  Contents: ");
    for item in ring.iter() {
        print!("{} ", item);
    }
    println!();
    
    // Pop and push to test wraparound
    let first = ring.pop().unwrap();
    println!("  Popped: {}", first);
    
    ring.push("Fourth").unwrap();
    println!("  Added 'Fourth' after pop");
    
    print!("  New contents: ");
    for item in ring.iter() {
        print!("{} ", item);
    }
    println!();
    
    // Memory pool demonstration
    let mut pool: MemoryPool<i32> = MemoryPool::new(4, 2);
    
    println!("\nMemory pool operations:");
    println!("  Total capacity: {}", pool.total_capacity());
    println!("  Available slots: {}", pool.available_slots());
    
    // Allocate some memory
    let ptr1 = pool.allocate().unwrap();
    let ptr2 = pool.allocate().unwrap();
    let ptr3 = pool.allocate().unwrap();
    
    unsafe {
        *ptr1 = 42;
        *ptr2 = 84;
        *ptr3 = 126;
    }
    
    println!("  After allocating 3 slots: {} available", pool.available_slots());
    
    unsafe {
        println!("  Values: {} {} {}", *ptr1, *ptr2, *ptr3);
    }
    
    // Deallocate
    pool.deallocate(ptr2);
    println!("  After deallocating one: {} available", pool.available_slots());
    
    // Safe pointer wrapper
    let mut data = [1, 2, 3, 4, 5];
    let mut safe_ptr = SafePtr::new(&mut data);
    
    println!("\nSafe pointer operations:");
    println!("  Length: {}", safe_ptr.len());
    
    // Safe access
    if let Some(value) = safe_ptr.get(2) {
        println!("  safe_ptr[2] = {}", value);
    }
    
    // Safe mutation
    if let Some(value) = safe_ptr.get_mut(2) {
        *value = 99;
        println!("  Modified safe_ptr[2] to 99");
    }
    
    // Bounds checking
    if safe_ptr.get(10).is_none() {
        println!("  safe_ptr[10] is out of bounds (correctly rejected)");
    }
    
    // Unsafe but documented access
    unsafe {
        let value = safe_ptr.get_unchecked(1);
        println!("  Unsafe access safe_ptr[1] = {}", value);
    }
    
    println!("\n🛡️  Safety patterns demonstrated:");
    println!("  ✅ Bounds checking in safe wrappers");
    println!("  ✅ Proper memory management with Drop");
    println!("  ✅ Clear unsafe function documentation");
    println!("  ✅ Invariant maintenance across operations");
    println!("  ✅ Safe abstractions over unsafe primitives");
}

safe_unsafe_patterns_demo();

---

## ⚠️ Common Pitfalls: Macros & Unsafe Code

### 1. Macro Hygiene Issues
```rust
macro_rules! bad_macro {
    () => {
        let x = 5;  // ⚠️ Could shadow user's variable
    };
}

let x = 10;
bad_macro!();
println!("{}", x);  // Which x?
```
**Solution:** Use unique names or accept parameters:
```rust
macro_rules! better_macro {
    ($var:ident) => {
        let $var = 5;
    };
}
```

### 2. Macro Expansion Order Confusion
```rust
macro_rules! debug {
    ($e:expr) => {
        println!("{} = {:?}", stringify!($e), $e);
    };
}

let mut x = 5;
debug!(x += 1);  // ⚠️ Evaluates x += 1, then tries to print it
```
**Note:** Expressions are evaluated before being passed to macros

### 3. Unsafe Block Too Large
```rust
// ❌ Bad: large unsafe block makes it hard to verify safety
unsafe {
    let ptr = some_ptr();
    // 50 lines of code...
    *ptr = 10;
    // More code...
}
```
**Solution:** Minimize unsafe blocks:
```rust
let ptr = some_ptr();
// Safe code...
unsafe {
    *ptr = 10;  // ✅ Only unsafe operation in block
}
// More safe code...
```

### 4. Dereferencing Null or Dangling Pointers
```rust
let ptr: *const i32 = std::ptr::null();
unsafe {
    let value = *ptr;  // ❌ Undefined behavior: dereferencing null
}
```
**Solution:** Always check pointer validity:
```rust
if !ptr.is_null() {
    unsafe {
        let value = *ptr;  // ✅ Checked first
    }
}
```

### 5. Macro Recursion Limits
```rust
macro_rules! recursive {
    () => {};
    ($x:tt $($rest:tt)*) => {
        recursive!($($rest)*);
    };
}

recursive!(a b c d e f g h i j k l m n o p q r s t u v w x y z ...);
// ❌ Error: recursion limit reached
```
**Solution:** Use `#![recursion_limit = "256"]` or redesign macro

### 6. Forgetting to Mark Functions as Unsafe
```rust
// ❌ Bad: function has unsafe operations but not marked unsafe
fn dangerous(ptr: *const i32) -> i32 {
    unsafe { *ptr }  // Unsafe operation hidden
}
```
**Solution:** Mark function as unsafe if it has safety requirements:
```rust
unsafe fn dangerous(ptr: *const i32) -> i32 {
    *ptr  // ✅ Caller knows this is unsafe
}
```

### 7. Data Races with Raw Pointers
```rust
let mut data = 5;
let ptr = &mut data as *mut i32;

unsafe {
    *ptr = 10;
}
data = 20;  // ⚠️ Potential aliasing violation
```
**Note:** Raw pointers bypass Rust's aliasing rules; be extremely careful

### 8. Macro Pattern Matching Ambiguity
```rust
macro_rules! ambiguous {
    ($x:expr) => { println!("expr") };
    ($x:ty) => { println!("type") };  // ❌ Can't distinguish at call site
}
```
**Solution:** Use different macro names or require explicit syntax

### 9. Memory Leaks with Forget
```rust
let boxed = Box::new(vec![1, 2, 3]);
std::mem::forget(boxed);  // ⚠️ Memory leaked, Drop not called
```
**Note:** `forget` is safe but can leak resources. Use only when necessary (e.g., FFI)

### 10. Transmute Misuse
```rust
unsafe {
    let x: f32 = 1.0;
    let y: u32 = std::mem::transmute(x);  // ⚠️ Reinterprets bits
}
```
**Danger:** `transmute` is extremely unsafe. Prefer:
- `as` for numeric conversions
- `from_bits` / `to_bits` for float/int conversions
- Proper type conversions when possible

**📚 Rust Book References:**
- [Chapter 19.5 - Macros](https://doc.rust-lang.org/book/ch19-06-macros.html)
- [Chapter 19.1 - Unsafe Rust](https://doc.rust-lang.org/book/ch19-01-unsafe-rust.html)
- [The Rustonomicon - Advanced Unsafe](https://doc.rust-lang.org/nomicon/)

---

# Part 2: Content from Second Lesson

---


In [ ]:
// Benchmarking and profiling techniques

use std::time::{Duration, Instant};
use std::collections::HashMap;

// Simple benchmarking macro
macro_rules! benchmark {
    ($name:expr, $iterations:expr, $code:block) => {
        {
            let start = Instant::now();
            
            for _ in 0..$iterations {
                let _ = $code;
            }
            
            let duration = start.elapsed();
            let avg_duration = duration / $iterations;
            
            println!("📊 {}: {} iterations", $name, $iterations);
            println!("   Total: {:?}", duration);
            println!("   Average: {:?}", avg_duration);
            println!("   Rate: {:.0} ops/sec", 
                    $iterations as f64 / duration.as_secs_f64());
            
            (duration, avg_duration)
        }
    };
}

// Performance comparison macro
macro_rules! compare_performance {
    ($iterations:expr, $(
        $name:expr => $code:block
    ),* $(,)?) => {
        {
            println!("\n🏁 Performance Comparison ({} iterations):", $iterations);
            let mut results = Vec::new();
            
            $(
                let (total, avg) = benchmark!($name, $iterations, $code);
                results.push(($name, total, avg));
                println!();
            )*
            
            // Find fastest
            if let Some((fastest_name, fastest_time, _)) = results.iter().min_by_key(|(_, total, _)| *total) {
                println!("🏆 Fastest: {} ({:?})", fastest_name, fastest_time);
                
                for (name, time, _) in &results {
                    if name != fastest_name {
                        let ratio = time.as_nanos() as f64 / fastest_time.as_nanos() as f64;
                        println!("   {} is {:.2}x slower", name, ratio);
                    }
                }
            }
        }
    };
}

// Different string concatenation approaches
fn concat_with_push_str(strings: &[&str]) -> String {
    let mut result = String::new();
    for s in strings {
        result.push_str(s);
    }
    result
}

fn concat_with_capacity(strings: &[&str]) -> String {
    let total_len: usize = strings.iter().map(|s| s.len()).sum();
    let mut result = String::with_capacity(total_len);
    for s in strings {
        result.push_str(s);
    }
    result
}

fn concat_with_join(strings: &[&str]) -> String {
    strings.join("")
}

fn concat_with_collect(strings: &[&str]) -> String {
    strings.iter().copied().collect()
}

// Vector operations comparison
fn vec_push_individual(n: usize) -> Vec<i32> {
    let mut vec = Vec::new();
    for i in 0..n {
        vec.push(i as i32);
    }
    vec
}

fn vec_with_capacity(n: usize) -> Vec<i32> {
    let mut vec = Vec::with_capacity(n);
    for i in 0..n {
        vec.push(i as i32);
    }
    vec
}

fn vec_collect_range(n: usize) -> Vec<i32> {
    (0..n as i32).collect()
}

fn vec_from_iter(n: usize) -> Vec<i32> {
    Vec::from_iter(0..n as i32)
}

fn benchmarking_demo() {
    println!("=== Benchmarking and Profiling ===");
    
    // String concatenation comparison
    let test_strings = vec!["Hello", " ", "world", "!", " ", "This", " ", "is", " ", "a", " ", "test", "."];
    
    compare_performance!(10000,
        "push_str" => {
            concat_with_push_str(&test_strings)
        },
        "with_capacity" => {
            concat_with_capacity(&test_strings)
        },
        "join" => {
            concat_with_join(&test_strings)
        },
        "collect" => {
            concat_with_collect(&test_strings)
        },
    );
    
    // Vector creation comparison
    let n = 1000;
    
    compare_performance!(1000,
        "push_individual" => {
            vec_push_individual(n)
        },
        "with_capacity" => {
            vec_with_capacity(n)
        },
        "collect_range" => {
            vec_collect_range(n)
        },
        "from_iter" => {
            vec_from_iter(n)
        },
    );
}

benchmarking_demo();